## 练习

1. 如果将转置卷积层改用Xavier随机初始化，结果有什么变化？
1. 调节超参数，能进一步提升模型的精度吗？
1. 预测测试图像中所有像素的类别。
1. 最初的全卷积网络的论文中 :cite:`Long.Shelhamer.Darrell.2015`还使用了某些卷积神经网络中间层的输出。试着实现这个想法。

In [ ]:
## 练习1
%matplotlib inline
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

# 加载预训练的ResNet网络
pretrained_net = torchvision.models.resnet18(pretrained=True)

# 移除预训练网络的最后几层
net = nn.Sequential(*list(pretrained_net.children())[:-2])

# 输入一个随机张量，检查特征图大小
X = torch.rand(size=(1, 3, 320, 480))
print(net(X).shape)

# 添加一个1x1卷积层将通道数转换为VOC数据集类别数（21类）
num_classes = 21
net.add_module('final_conv', nn.Conv2d(512, num_classes, kernel_size=1))

# 添加转置卷积层
net.add_module('transpose_conv', nn.ConvTranspose2d(num_classes, num_classes,
                                                    kernel_size=64, padding=16, stride=32))

# 使用Xavier随机初始化转置卷积层的权重
net.transpose_conv.weight.data = torch.nn.init.xavier_uniform_(net.transpose_conv.weight.data)

# bilinear_kernel函数用于初始化转置卷积层
def bilinear_kernel(in_channels, out_channels, kernel_size):
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = (torch.arange(kernel_size).reshape(-1, 1),
          torch.arange(kernel_size).reshape(1, -1))
    filt = (1 - torch.abs(og[0] - center) / factor) * \
           (1 - torch.abs(og[1] - center) / factor)
    weight = torch.zeros((in_channels, out_channels, kernel_size, kernel_size))
    weight[range(in_channels), range(out_channels), :, :] = filt
    return weight

# 初始化转置卷积层
W = bilinear_kernel(num_classes, num_classes, 64)
net.transpose_conv.weight.data.copy_(W)

# 加载数据集
batch_size, crop_size = 32, (320, 480)
train_iter, test_iter = d2l.load_data_voc(batch_size, crop_size)

# 定义损失函数
def loss(inputs, targets):
    return nn.functional.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)

# 定义训练配置
num_epochs, lr, wd, devices = 5, 0.001, 1e-3, d2l.try_all_gpus()
trainer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd)

# 训练模型
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

torch.Size([1, 512, 10, 15])


In [ ]:
## 练习2
%matplotlib inline
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

# 加载预训练的ResNet网络
pretrained_net = torchvision.models.resnet18(pretrained=True)

# 移除预训练网络的最后几层
net = nn.Sequential(*list(pretrained_net.children())[:-2])

# 输入一个随机张量，检查特征图大小
X = torch.rand(size=(1, 3, 320, 480))
print(net(X).shape)

# 添加一个1x1卷积层将通道数转换为VOC数据集类别数（21类）
num_classes = 21
net.add_module('final_conv', nn.Conv2d(512, num_classes, kernel_size=1))

# 添加转置卷积层
net.add_module('transpose_conv', nn.ConvTranspose2d(num_classes, num_classes,
                                                    kernel_size=64, padding=16, stride=32))

# 使用双线性核初始化转置卷积层
W = bilinear_kernel(num_classes, num_classes, 64)
net.transpose_conv.weight.data.copy_(W)

# 加载数据集
batch_size, crop_size = 32, (320, 480)
train_iter, test_iter = d2l.load_data_voc(batch_size, crop_size)

# 定义损失函数
def loss(inputs, targets):
    return nn.functional.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)

# 调整超参数
num_epochs, lr, wd = 10, 0.01, 5e-4  # 调整学习率和权重衰减
devices = d2l.try_all_gpus()
trainer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd)

# 训练模型
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

In [ ]:
## 练习3
%matplotlib inline
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

# 加载预训练的ResNet网络
pretrained_net = torchvision.models.resnet18(pretrained=True)

# 移除预训练网络的最后几层
net = nn.Sequential(*list(pretrained_net.children())[:-2])

# 添加一个1x1卷积层将通道数转换为VOC数据集类别数（21类）
num_classes = 21
net.add_module('final_conv', nn.Conv2d(512, num_classes, kernel_size=1))

# 添加转置卷积层
net.add_module('transpose_conv', nn.ConvTranspose2d(num_classes, num_classes,
                                                    kernel_size=64, padding=16, stride=32))

# 使用双线性核初始化转置卷积层
W = bilinear_kernel(num_classes, num_classes, 64)
net.transpose_conv.weight.data.copy_(W)

# 加载数据集
batch_size, crop_size = 32, (320, 480)
train_iter, test_iter = d2l.load_data_voc(batch_size, crop_size)

# 定义损失函数
def loss(inputs, targets):
    return nn.functional.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)

# 训练模型
num_epochs, lr, wd, devices = 5, 0.001, 1e-3, d2l.try_all_gpus()
trainer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd)
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

# 预测函数
def predict(img):
    X = test_iter.dataset.normalize_image(img).unsqueeze(0)
    pred = net(X.to(devices[0])).argmax(dim=1)
    return pred.reshape(pred.shape[1], pred.shape[2])

# 预测测试图像中所有像素的类别
for X in test_images:
    pred = predict(X)
    # 这里可以显示或处理预测的结果
    # 比如: d2l.show_images(X, pred, ...)

# label2image函数用于将预测的类别转换为颜色图像
def label2image(pred):
    colormap = torch.tensor(d2l.VOC_COLORMAP, device=devices[0])
    X = pred.long()
    return colormap[X, :]

In [ ]:
%matplotlib inline
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

# 加载预训练的ResNet网络
pretrained_net = torchvision.models.resnet18(pretrained=True)

# 基础网络使用预训练网络的前几层
class ResNet18(nn.Module):
    def __init__(self, pretrained_net):
        super(ResNet18, self).__init__()
        self.stage1 = nn.Sequential(*list(pretrained_net.children())[:4])  # 第一阶段
        self.stage2 = nn.Sequential(*list(pretrained_net.children())[4:5]) # 第二阶段
        self.stage3 = nn.Sequential(*list(pretrained_net.children())[5:6]) # 第三阶段
        self.stage4 = nn.Sequential(*list(pretrained_net.children())[6:7]) # 第四阶段
        self.stage5 = nn.Sequential(*list(pretrained_net.children())[7:-2]) # 第五阶段

    def forward(self, x):
        x1 = self.stage1(x)
        x2 = self.stage2(x1)
        x3 = self.stage3(x2)
        x4 = self.stage4(x3)
        x5 = self.stage5(x4)
        return x1, x2, x3, x4, x5

# 创建基础网络实例
base_net = ResNet18(pretrained_net)

# 全卷积网络
class FCN(nn.Module):
    def __init__(self, base_net, num_classes):
        super(FCN, self).__init__()
        self.base_net = base_net
        self.num_classes = num_classes
        # 1x1卷积用于降低通道数
        self.conv1 = nn.Conv2d(64, num_classes, kernel_size=1)
        self.conv2 = nn.Conv2d(128, num_classes, kernel_size=1)
        self.conv3 = nn.Conv2d(256, num_classes, kernel_size=1)
        self.conv4 = nn.Conv2d(512, num_classes, kernel_size=1)
        self.conv5 = nn.Conv2d(512, num_classes, kernel_size=1)
        # 转置卷积
        self.trans_conv = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=64, padding=16, stride=32)

    def forward(self, x):
        x1, x2, x3, x4, x5 = self.base_net(x)
        x1 = self.conv1(x1)
        x2 = self.conv2(x2)
        x3 = self.conv3(x3)
        x4 = self.conv4(x4)
        x5 = self.conv5(x5)
        # 融合不同阶段的输出
        x = x1 + x2 + x3 + x4 + x5
        return self.trans_conv(x)

# 创建全卷积网络实例
net = FCN(base_net, num_classes=21)

# 设置超参数
num_epochs, lr, wd, batch_size, crop_size = 5, 0.001, 1e-3, 32, (320, 480)
devices = d2l.try_all_gpus()

# 定义损失函数
loss = nn.CrossEntropyLoss(reduction='none')

# 加载数据集
train_iter, test_iter = d2l.load_data_voc(batch_size, crop_size)

# 初始化网络参数
def init_weights(m):
    if type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.ConvTranspose2d:
        nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)

# 创建优化器
trainer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd)

# 训练模型
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

# 预测函数
def predict(img):
    X = test_iter.dataset.normalize_image(img).unsqueeze(0)
    pred = net(X.to(devices[0])).argmax(dim=1)
    return pred.reshape(pred.shape[1], pred.shape[2])

# 预测并显示结果
def label2image(pred):
    colormap = torch.tensor(d2l.VOC_COLORMAP, device=devices[0])
    X = pred.long()
    return colormap[X, :]

voc_dir = d2l.download_extract('voc2012', 'VOCdevkit/VOC2012')
test_images, test_labels = d2l.read_voc_images(voc_dir, False)
n, imgs = 4, []
for i in range(n):
    crop_rect = (0, 0, 320, 480)
    X = torchvision.transforms.functional.crop(test_images[i], *crop_rect)
    pred = label2image(predict(X))
    imgs += [X.permute(1, 2, 0), pred.cpu(),
             torchvision.transforms.functional.crop(
                 test_labels[i], *crop_rect).permute(1, 2, 0)]
d2l.show_images(imgs[::3] + imgs[1::3] + imgs[2::3], 3, n, scale=2)